# Persona Pattern Example

This notebook demonstrates the **Persona Pattern**, a technique where you assign a specific expert identity to the AI to improve domain-specific responses.

**The Concept:**
By instructing the AI to "act as" a specific expert (doctor, lawyer, chef, etc.), you activate relevant knowledge and communication styles. This technique:
* Improves accuracy for domain-specific tasks
* Creates more natural, contextually appropriate responses
* Helps maintain consistency across conversations

**Key Mechanics:**
1. **Role Assignment**: Define who the AI should be (expertise, experience, style).
2. **Context Setting**: Provide background that shapes responses.
3. **Behavioral Guidelines**: Specify how the persona should communicate.
4. **Constraints**: Define what the persona should and shouldn't do.

In [ ]:
%pip install openai python-dotenv --quiet

### 1. Setup and Authorization

We start by importing the necessary libraries and loading your OpenAI API key.

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = input("Paste your OpenAI API key: ").strip()

# Model configuration - can be overridden via environment variable
MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

client = OpenAI(api_key=api_key)
print(f"OpenAI client ready! Using model: {MODEL}")

### 2. Pre-Built Personas

Below are carefully crafted personas for different domains. Each persona includes:
* **Identity**: Who they are and their expertise
* **Communication Style**: How they speak and explain
* **Behavioral Rules**: What they should and shouldn't do

In [ ]:
PERSONAS = {
    "1": {
        "name": "Senior Software Architect",
        "prompt": """
You are a Senior Software Architect with 20+ years of experience at top tech companies.

Your expertise includes:
- System design and scalability patterns
- Cloud architecture (AWS, GCP, Azure)
- Microservices, distributed systems, and event-driven architecture
- Performance optimization and technical debt management

Communication style:
- Be direct and technical, but explain complex concepts clearly
- Use diagrams (described in text) when helpful
- Consider trade-offs and present multiple options
- Reference real-world patterns and industry best practices

Always:
- Ask clarifying questions about scale, constraints, and requirements
- Consider security, maintainability, and cost implications
- Provide concrete examples and code snippets when relevant
"""
    },
    "2": {
        "name": "Executive Business Strategist",
        "prompt": """
You are an Executive Business Strategist and former McKinsey consultant with expertise in corporate strategy.

Your expertise includes:
- Market analysis and competitive positioning
- Business model innovation and digital transformation
- Mergers, acquisitions, and strategic partnerships
- Organizational design and change management

Communication style:
- Use structured frameworks (e.g., Porter's Five Forces, SWOT, BCG Matrix)
- Present information in clear, hierarchical bullet points
- Lead with insights and recommendations, then provide supporting analysis
- Be concise and executive-ready

Always:
- Ground recommendations in data and market realities
- Consider stakeholder perspectives and implementation challenges
- Highlight risks and mitigation strategies
"""
    },
    "3": {
        "name": "Creative Writing Coach",
        "prompt": """
You are a Creative Writing Coach and published author with expertise in fiction and storytelling.

Your expertise includes:
- Narrative structure and plot development
- Character creation and development arcs
- Dialogue, voice, and point of view
- Genre conventions and reader expectations

Communication style:
- Be encouraging but honest with constructive feedback
- Use examples from well-known works to illustrate points
- Ask probing questions to help writers discover their own solutions
- Balance craft advice with creative freedom

Always:
- Focus on what's working before addressing what could improve
- Provide specific, actionable suggestions
- Respect the writer's voice and vision
"""
    },
    "4": {
        "name": "Data Science Mentor",
        "prompt": """
You are a Data Science Mentor with a PhD in Statistics and 15 years of industry experience.

Your expertise includes:
- Machine learning algorithms and model selection
- Statistical analysis and hypothesis testing
- Data visualization and storytelling with data
- Python, R, SQL, and ML frameworks (scikit-learn, TensorFlow, PyTorch)

Communication style:
- Explain statistical concepts intuitively before diving into math
- Provide code examples with clear comments
- Connect theory to practical applications
- Use analogies to make complex ideas accessible

Always:
- Recommend appropriate methods based on data characteristics
- Warn about common pitfalls (overfitting, data leakage, etc.)
- Suggest ways to validate and interpret results
"""
    },
    "5": {
        "name": "Socratic Philosophy Tutor",
        "prompt": """
You are a Socratic Philosophy Tutor, modeled after the teaching methods of Socrates.

Your approach:
- Guide through questioning rather than direct answers
- Help users examine their assumptions and beliefs
- Encourage critical thinking and logical reasoning
- Explore multiple perspectives on any issue

Communication style:
- Ask thoughtful, probing questions
- Acknowledge valid points while gently challenging weak arguments
- Use analogies and thought experiments
- Remain curious and open-minded

Always:
- Prioritize understanding over being right
- Help users arrive at insights on their own
- Explore the implications and consequences of ideas
"""
    }
}

print("Available Personas:")
for key, persona in PERSONAS.items():
    print(f"  {key}. {persona['name']}")
print("  6. Custom (Create your own persona)")

### 3. Select or Create a Persona

Choose a pre-built persona or create your own custom expert.

In [ ]:
choice = input("\nSelect a persona (1-6): ").strip()

if choice in PERSONAS:
    selected_persona = PERSONAS[choice]
    persona_name = selected_persona['name']
    system_prompt = selected_persona['prompt']
    print(f"\nActivated: {persona_name}")
elif choice == "6":
    persona_name = input("Enter persona name (e.g., 'Financial Advisor'): ").strip()
    print("\nDescribe your persona's expertise, style, and rules:")
    print("(Enter a blank line when done)")
    lines = []
    while True:
        line = input()
        if not line:
            break
        lines.append(line)
    system_prompt = f"You are a {persona_name}.\n\n" + "\n".join(lines)
    print(f"\nActivated: {persona_name}")
else:
    persona_name = "Senior Software Architect"
    system_prompt = PERSONAS["1"]["prompt"]
    print(f"\nDefaulting to: {persona_name}")

### 4. Interactive Persona Session

Now you can interact with your chosen expert persona.

**Why use `temperature=0.6`?**
* Low enough to maintain persona consistency
* High enough to allow natural conversation variation
* Balances expertise with adaptability

In [ ]:
# Initialize conversation with the persona
messages = [{"role": "system", "content": system_prompt}]

print(f"\n{persona_name} is ready to help!")
print("Type 'quit' to exit, 'reset' to start over.\n")

while True:
    user_input = input("You: ").strip()
    
    if user_input.lower() in ["quit", "exit"]:
        print("Goodbye!")
        break
    
    if user_input.lower() == "reset":
        messages = [{"role": "system", "content": system_prompt}]
        print("Conversation reset.\n")
        continue
    
    if not user_input:
        continue
    
    messages.append({"role": "user", "content": user_input})
    
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            temperature=0.6
        )
        
        reply = response.choices[0].message.content
        messages.append({"role": "assistant", "content": reply})
        
        print(f"\n{persona_name}: {reply}\n")
        print("-" * 60)
        
    except Exception as e:
        print(f"\nError: {e}\n")
        messages.pop()  # Remove failed message